In [1]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
import numpy as np
import tensorflow as tf

In [2]:
from os import walk
from PIL import Image

path = 'figures'
data = {}
for (dirpath, dirnames, filenames) in walk(path):
    for dirname in dirnames:
        if dirname not in data:
            data[dirname] = []
        
        for (figpath, dontcare, figs) in walk(dirpath + '/' + dirname):
            for fig in figs:
                figImage = Image.open(figpath + '/' + fig).convert('L')    
                data[dirname].append(figImage.resize((28, 28), Image.ANTIALIAS))

In [4]:
def to2d(img):
    array = np.asarray(img)
    x = []
    for row in array:
        we = []
        x.append(we)
        for value in row:
            we.append(1-value/255)
    return x

In [5]:
def one_hot(data_set):
    i = 0
    output = len(data_set) + 1
    X = []
    Y = []
    Z = []
    for fig in data_set:   
        i = i + 1
        oh = np.zeros(output, dtype=np.float64)
        oh[i] = 1 if fig == 'circle' else 0
        for img in data_set[fig]:
            for r in range(5):
                X.append(to2d(img))
                Y.append(oh)
                Z.append(img)
    return np.array(X, dtype=np.float64).reshape([-1, 28, 28, 1]), np.array(Y, dtype=np.float64), Z

X,Y,Z = one_hot(data)

In [6]:
import random
shuf = list(zip(X, Y,Z))
random.shuffle(shuf)
X = [e[0] for e in shuf]
Y = [e[1] for e in shuf]
Z = [e[2] for e in shuf]

In [7]:
# batch = 50
# # graph = tf.Graph() 
# # with graph.as_default():
#     network = input_data(shape=[None, 28, 28, 1], name='input', dtype=tf.float32)
#     network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
#     network = max_pool_2d(network, 2)
#     network = local_response_normalization(network)
# #     network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
# #     network = max_pool_2d(network, 2)
# #     network = local_response_normalization(network)
# #     network = fully_connected(network, 128, activation='tanh')
# #     network = dropout(network, 0.8)
#     network = fully_connected(network, 256, activation='tanh')
#     network = dropout(network, 0.8)
#     network = fully_connected(network, 4, activation='softmax')
#     network = regression(network, optimizer='SGD', learning_rate=0.5,
#                              loss='categorical_crossentropy', name='target')
#     gmodel = tflearn.DNN(network, tensorboard_verbose=3, tensorboard_dir='board')
#     gmodel.fit({'input': X[:-batch]}, {'target': Y[:-batch]}, n_epoch=10,
#                        validation_set=({'input': X[-batch:]}, {'target': Y[-batch:]}),
#                        snapshot_step=100, show_metric=True, run_id='fig_net_51')
    
#     gmodel.evaluate({'input': X[:-batch]}, {'target': Y[:-batch]})

In [20]:
activations = ('relu','tanh')
optimizers  = ('sgd', 'adam')
losses      = ('categorical_crossentropy','whatever')
regulizers  = ('L1','L2')

def create_parameterized_network(params):
    name = "net_" + str(params).replace('(','').replace(')','').replace(',','').replace(' ','')
    print('Create network:' + name)
    network = input_data(shape=[None, 28, 28, 1], name='input', dtype=tf.float32)
    layer_n = 0
    for p in params[:-1]:
        layer_n = layer_n + 1
        layer_type = p[0] 
        shape = p[1]
        activation = activations[p[2]]
        regulizer = regulizers[p[3]]
        drpt = p[4]
        fltr = p[5]
        max_pool = p[6]
        
        if layer_type == 0:
            layer_name = 'fc_' + str(layer_n)
            print('Create layer {} with shape: {}, activation: {}, dropout: {}'
                  .format(layer_name, shape, activation, drpt))
            network = fully_connected(network, shape, 
                                      activation=activation, 
                                      name=layer_name)
            network = dropout(network, drpt)
        elif layer_type == 1:     
            layer_name = 'conv_' + str(layer_n)
            print('Create layer {} with shape: {}, activation: {}, regularizer: {}, filter: {}, max_pool: {}'
                  .format(layer_name, shape, activation, regulizer, fltr, max_pool))
            network = conv_2d(network, shape, fltr, 
                              activation=activation, 
                              regularizer=regulizer,
                              name=layer_name)
            network = max_pool_2d(network, p[6])
            network = local_response_normalization(network) 
    
    network = fully_connected(network, 4, activation='softmax', name='prob')
    
    optimizer=optimizers[params[-1][0]]
    learning_rate=params[-1][1]
    loss=losses[params[-1][2]]
    print('Add regression {} at {} with {}'.format(optimizer, learning_rate, loss)
                  .format(layer_name, shape, activation, regulizer, fltr, max_pool))
    network = regression(network, 
                         optimizer=optimizer, 
                         learning_rate=learning_rate,
                         loss=loss, 
                         name='target')
    
    return network, name

In [27]:
def learn(params, batch):
    graph = tf.Graph() 
    with graph.as_default():
            network, name = create_parameterized_network(params)
            gmodel = tflearn.DNN(network, tensorboard_verbose=3, tensorboard_dir='mnist/board')
            gmodel.fit({'input': X[:-batch]}, {'target': Y[:-batch]}, n_epoch=1,
                       validation_set=({'input': X[-batch:]}, {'target': Y[-batch:]}),
                       snapshot_step=100, show_metric=True, run_id=name)
            return gmodel.evaluate({'input': X[:-batch]}, {'target': Y[:-batch]})

In [28]:
print(learn((
    (1,32, 0,1,0,  3,2), 
    (1,64, 0,1,0,  3,2),
    (0,128,1,0,0.8,0,0),
    (0,256,1,0,0.8,0,0),
    (1, 0.01, 0)
), 50))

Training Step: 3  | total loss: 0.28904 | time: 2.636s
| Adam | epoch: 001 | loss: 0.28904 - acc: 0.2915 -- iter: 192/255
Training Step: 4  | total loss: 0.07428 | time: 4.285s
| Adam | epoch: 001 | loss: 0.07428 - acc: 0.3424 | val_loss: 0.00003 - val_acc: 0.2200 -- iter: 255/255
--


Exception in thread Thread-73:
Traceback (most recent call last):
  File "c:\users\etashkinov\appdata\local\programs\python\python35\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "c:\users\etashkinov\appdata\local\programs\python\python35\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\etashkinov\appdata\local\programs\python\python35\lib\site-packages\tflearn\data_flow.py", line 187, in fill_feed_dict_queue
    data = self.retrieve_data(batch_ids)
  File "c:\users\etashkinov\appdata\local\programs\python\python35\lib\site-packages\tflearn\data_flow.py", line 222, in retrieve_data
    utils.slice_array(self.feed_dict[key], batch_ids)
  File "c:\users\etashkinov\appdata\local\programs\python\python35\lib\site-packages\tflearn\utils.py", line 180, in slice_array
    return [x[start] for x in X]
  File "c:\users\etashkinov\appdata\local\programs\python\python35\lib\site-packages\tflearn\utils.py", line 180, in <li

[0.0]


In [26]:
np.array(X).shape

(305, 28, 28, 1)

In [29]:
np.array(Y).shape

(305, 4)